<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generate Readme for Awesome Notebooks

## Input

### Import librairies

In [123]:
import os
import requests
import urllib.parse
import pandas as pd
import copy
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [136]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"

# welcome variables
welcome_template = "Welcome_template.ipynb"
welcome = "Welcome.ipynb"
replace_var_quote = f'"[[DYNAMIC_LIST]]",\n'



# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/unskript/Awesome-CloudOps-Automation/tree/master'
local_OSS_url = 'http://127.0.0.1:8888/lab/tree'
#fix these!
github_download_url = 'https://raw.githubusercontent.com/unskript/Awesome-CloudOps-Automation/master/'
naas_download_url ='https://app.naas.ai/user-redirect/naas/downloader?url='
unSkript_logo ='https://unskript.com/assets/favicon.png'

#update these!!
template_request = "https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+"
bug_report = "https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title="

### Get files list

In [137]:
repo = Repo('.')
branch = repo.active_branch
list_of_dir = f"https://api.github.com/repos/unskript/Awesome-CloudOps-Automation/git/trees/{branch.name}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
notebooks = []

for file in r_gh:
    if ".github" not in file.get("path") and ".gitignore" not in file.get("path") and "templates" not in file.get("path") and "/" in file.get("path"):
        if file.get("path").endswith(".ipynb"):
            temp = file.get("path").split("/")
            if temp == -1:
                data = {
                    "root": None,
                    "subdir": file.get("path")
                }
                notebooks.append(data)
            else:
                last_folder = ""
                file_name = temp[-1]
                temp.pop()
                for folder in temp:
                    last_folder += "/" + folder
                root = last_folder[1:]
                data = {
                    "root": root,
                    "subdir": file_name
                }
                notebooks.append(data)

df_github = pd.DataFrame(notebooks)
df_github

,root,subdir
0,AWS,AWS_Access_Key_Rotation.ipynb
1,AWS,Add-new-IAM-user.ipynb
2,AWS,Configure-url-endpoint-on-a-cloudwatch-alarm.i...
3,AWS,Delete_Unattached_EBS_Volume.ipynb
4,AWS,Detach-Instance-from-ASG-and-Load-Balancer.ipynb
5,AWS,Detect-ECS-failed-deployment.ipynb
6,AWS,EC2-Disk-Cleanup.ipynb
7,AWS,Enforce-HTTP-Redirection-across-AWS-ALB.ipynb
8,AWS,Enforce-Mandatory-Tags-Across-All-AWS-Resource...
9,AWS,Find-EC2-Instances-Scheduled-to-retire.ipynb


## Model

### Reformat functions

In [138]:
def reformat_file_name(file):
    file_nice = file.replace('_', ' ')
    file_nice = file_nice.replace('-', ' ')
    file_nice = file_nice.replace(notebook_ext, '')
    file_nice = file_nice.strip()
    if (file_nice != ""):
            file_nice = file_nice[0].capitalize() + file_nice[1:]
    return file_nice

### Get functions

In [139]:
def get_open_button(download_link, title_url):
    return f"""<a href="{download_link}" target="_new"><img src="{unSkript_logo}"/></a><br><br><a href="{template_request}">💡 Template request</a> | <a href="{bug_report}{title_url}:+Error+short+description">🚨 Bug report</a>"""

#title = get_title(folder_nice, file, github_url, local_url)
def get_title(folder_nice, file,file_nice, github_runbook_url, local_url):
    title_url = (f"{folder_nice} - {file}").replace(" ", "+")
#    return f"""# {folder_nice} - {file_nice}\n{get_open_button(download_link, title_url)}"""
    return f'| {folder_nice} | [{file_nice}]({github_runbook_url}) | [Open in browser]({local_url})'

def get_tags(text):
    result = []
    tags = text.split(' ')
    for tag in tags:
        if len(tag) >= 2 and tag[0] == '#' and tag[1] != ' ' and tag[1] != '#':
            result.append(tag)
    return result

### Convert filepath in Markdown text

In [140]:
def get_file_md(folder_nice, folder_url, files, json_templates, title_sep="##", subtitle_sep="*"):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                title = f"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}\n"
                
                md += title

   # print("md", md)
   # print(json_templates)
    return md

In [141]:
def get_file_md_quote(folder_nice, folder_url, files, json_templates, title_sep="##", subtitle_sep="*"):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                title = f'"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}",\n'
                
                md += title

   # print("md", md)
   # print(json_templates)
    return md

### Generate markdown for each notebooks

In [142]:
generated_list = ""
generated_list_quote = ""
json_templates = []
list_of_tools = []
index_max = len(notebooks)
index = 0

while index <= (index_max) - 1:
    folder_nice = notebooks[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        md_round = ""
        md_round_quote=""
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
        print(folder_nice)
        while True:
            if notebooks[index].get("root") != folder_nice:
                break
            print(notebooks[index].get("subdir"))
            files.append(notebooks[index].get("subdir"))
            index += 1
            if index == index_max:
                break
        if ("/" not in folder_nice):
            md_round += get_file_md(folder_nice, folder_url, files, json_templates)
            md_round_quote += get_file_md_quote(folder_nice, folder_url, files, json_templates)
        else:
            folder_url = urllib.parse.quote(folder_nice)
            subfolder_nice = folder_nice.split('/')[1].replace('_', ' ').replace(folder_nice, '').strip()
            #md_round += f'"{get_title(folder_nice, file, files, github_url, local_OSS_url)}"'
            #md_round += get_file_md(subfolder_nice, folder_url, files, json_templates, "\t###", "\t-")
            
        print("md_round", md_round)
        print("md_round_quote", md_round_quote)  
        generated_list += md_round
        generated_list_quote += md_round_quote

AWS
AWS_Access_Key_Rotation.ipynb
Add-new-IAM-user.ipynb
Configure-url-endpoint-on-a-cloudwatch-alarm.ipynb
Delete_Unattached_EBS_Volume.ipynb
Detach-Instance-from-ASG-and-Load-Balancer.ipynb
Detect-ECS-failed-deployment.ipynb
EC2-Disk-Cleanup.ipynb
Enforce-HTTP-Redirection-across-AWS-ALB.ipynb
Enforce-Mandatory-Tags-Across-All-AWS-Resources.ipynb
Find-EC2-Instances-Scheduled-to-retire.ipynb
Get-Aws-Elb-Unhealthy-Instances.ipynb
Lowering-AWS-CloudTrail-Costs-by-Removing-Redundant-Trails.ipynb
Monitor-AWS-DynamoDB-provision-capacity.ipynb
Notify-about-unused-keypairs.ipynb
Publicly-Accessible-Amazon-RDS-Instances.ipynb
Remediate-unencrypted-S3-buckets.ipynb
Renew_SSL_Certificate.ipynb
Resize-EBS-Volume.ipynb
Resize-List-Of-Pvcs.ipynb
Resize_PVC.ipynb
Restart-Aws-Instance-given-Tag.ipynb
Restart-Unhealthy-Services-Target-Group.ipynb
Restart_AWS_EC2_Instances.ipynb
Restrict-S3-Buckets-with-READ-WRITE-Permissions.ipynb
Run-EC2-from-AMI.ipynb
Secure-Publicly-accessible-Amazon-RDS-Snapshot.i

## Output

### Preview the generated list

### Generate readme for github repository

In [145]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, generated_list)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()

### Generate welcome page for OSS

In [147]:
#todo